## Modeling with a Dataframe

After webscraping all of the Euros data and formatting into a dataframe then csv in my [euros_scraping.ipynb](http://localhost:8888/files/Documents/Coding/Notebooks/repos/euros-WebScraping/euros_scraping.ipynb?_xsrf=2%7C8c3fecd8%7C02d37b9b4c7c7af0dd7bca94af94aa7f%7C1716421226) notebook, I will now attempt to perform Modeling and predictions

THe hope is I scraped the relevant data, having all results recorded from Euro's and Euro's qualifiers into the csv.

In this notebook, I will use pandas as the main method of modeling this data.

In [2]:
import pandas as pd

In [3]:
matches = pd.read_csv("euros.csv", index_col=0)
matches["mDate"] = pd.to_datetime(matches["mDate"], format="%d.%m.%y")
matches.loc[matches["mDate"].dt.year > 2024, "mDate"] -= pd.DateOffset(years=100) 

# necesssary to offset pre-2000 mDates being interpreted as 20%yy (10.05.59 as 1959)

In [4]:
matches["hTeam"].value_counts().head()

hTeam
Italy       84
Germany     84
Spain       83
Denmark     81
Portugal    80
Name: count, dtype: int64

In [5]:
matches["aTeam"].value_counts().head()

aTeam
Spain          88
Italy          79
Netherlands    77
Denmark        76
Portugal       76
Name: count, dtype: int64

### Next Steps 

Currently, a single National team's data is split between home and away sections. For example, when we scraped the page data, sometimes Italy was listed as the left team and sometimes it was listed as the right team.

For brevity, I selected to set Italy as the home team whenever it appeared on the left and designated it as the away team when it appeared on the right.

It will be better to reorganize the data, coalescing all of Italy's matches together, re-stiching the data from

> hTeam --> team <BR>
> hGoals --> gScored <BR>
> hPenalties --> pScored <BR>
> hResult --> result <BR>
> aTeam --> opponent <BR>
> aGoals --> gAllowed <BR>
> aPenalties --> pAllowed <BR>

This will be difficult and require duplicate row data <br>
(i.e Italy as home team, Denmark as away, recording same match data for both in their own "team" as opposite's "opponent" so we can focus on a Nation's individual performance)

Instead of manipulating our current dataframe, it may be best to update into a new dataframe

In [6]:
# comp | mType | team | opponent | result | goals | gAllowed | penalties | pAllowed | mDate
mInfo = {"team": ["filler"], "opponent": ["fller"],
        "result": ['f'], "goals": [1.1], "gAllowed": [1.1], 
        "penalties": [1.1], "pAllowed": [1.1], "mDate": [pd.NA], 
        'mType': ["filler"], 'comp': ["filler"],}
cMatches = pd.DataFrame(data=mInfo)
cMatches[["mDate"]] = cMatches[["mDate"]].astype("datetime64[ns]")
cMatches = cMatches.drop(index=0)
cMatches

,team,opponent,result,goals,gAllowed,penalties,pAllowed,mDate,mType,comp


In [7]:
for index, row in matches.iterrows():
    cMatches.loc[-1] = [row["hTeam"], row["aTeam"], row["hResult"], row["hGoals"], row["aGoals"], row["hPenalties"], row["aPenalties"], row["mDate"], row["mType"], row["comp"], ]
    cMatches.index += 1
    cMatches.loc[-1] = [row["aTeam"], row["hTeam"], row["aResult"], row["aGoals"], row["hGoals"], row["aPenalties"], row["hPenalties"], row["mDate"], row["mType"], row["comp"]]
    cMatches.index += 1
cMatches.index = list(range(len(cMatches["team"])))

In [8]:
cMatches.head()

,team,opponent,result,goals,gAllowed,penalties,pAllowed,mDate,mType,comp
0,Georgia,Greece,W,0,0,4.0,2.0,2024-03-26,Final,"Euros 2024, Qualifiers"
1,Greece,Georgia,L,0,0,2.0,4.0,2024-03-26,Final,"Euros 2024, Qualifiers"
2,Wales,Poland,L,0,0,4.0,5.0,2024-03-26,Final,"Euros 2024, Qualifiers"
3,Poland,Wales,W,0,0,5.0,4.0,2024-03-26,Final,"Euros 2024, Qualifiers"
4,Ukraine,Iceland,W,2,1,NaN,NaN,2024-03-26,Final,"Euros 2024, Qualifiers"


In [9]:
cMatches.tail()

,team,opponent,result,goals,gAllowed,penalties,pAllowed,mDate,mType,comp
6089,Denmark,ČSSR,L,1,5,NaN,NaN,1959-10-18,Round of 16,Euros 1960
6090,Ireland,ČSSR,W,2,0,NaN,NaN,1959-04-05,Qualifying Round,Euros 1960
6091,ČSSR,Ireland,L,0,2,NaN,NaN,1959-04-05,Qualifying Round,Euros 1960
6092,ČSSR,Ireland,W,4,0,NaN,NaN,1959-05-10,Qualifying Round,Euros 1960
6093,Ireland,ČSSR,L,0,4,NaN,NaN,1959-05-10,Qualifying Round,Euros 1960


In [10]:
cMatches["team"].value_counts().head()

team
Spain          171
Italy          163
Denmark        157
Portugal       156
Netherlands    156
Name: count, dtype: int64

## Setting category predictors

Below, we are adding additional columns as metrics for predicting a match. A team may perform differently based on the match type of the competition and their competitor 

The target column is if a team achieved its desired result, 

> a "W" was desired, equating to a 1 <br>
> a "L" or "D" was not desired, equating to a 0

In [38]:
cMatches["oppCode"] = cMatches["opponent"].astype("category").cat.codes
cMatches["mCode"] = cMatches["mType"].astype("category").cat.codes

In [39]:
cMatches["target"] = (cMatches["result"] == "W").astype("int")

## Predictions 
With a RandomForestClassifer, we can run simulations with our category codes non-linearity,

We then split from a "trained" and "test" dataframe 

> A Train dataframe to . . . train our rf model on our selected predictors (opponent and match type) to see what the expected result (target) would be in our test model <br>
> A Test data frame to see how our predictions actually compare with that of the outcome 

In [57]:
from sklearn.ensemble import RandomForestClassifier

# associates non-linearity 
# i.e oppCode 22 doesn't imply numerical difference between oppCode 23
# just categorical differences

rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

train = cMatches[cMatches["mDate"] < "2000-01-01"] 
test = cMatches[cMatches["mDate"] > "2000-01-01"]

predictors = ["oppCode", "mCode"]

rf.fit(train[predictors], train["target"])
preds = rf.predict(test[predictors])

## Comparisons

With our trained rf, we then create predictions based on the same metrics for the test data set

Using the accuracy_score, we're able to see how often our trained model guessed correctlty

In [58]:
from sklearn.metrics import accuracy_score

# measures our predictions 

acc = accuracy_score(test["target"], preds)
acc

0.6699886104783599

In [61]:
combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,1632,433
1,726,721


In [63]:
from sklearn.metrics import precision_score

precision_score(test["target"], preds)

0.6247833622183708

## Refining Precisions

On a rolling average